In [1]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import pymesh
import meshio
import math as m

from mesh3d import Mesh3D

%matplotlib inline
plt.rcParams["figure.figsize"] = 12.8, 9.6

In [2]:
CELL_SIZE=5e-1

In [3]:
def cart2sph(x,y,z):
    XsqPlusYsq = x**2 + y**2
    r = m.sqrt(XsqPlusYsq + z**2)               # r
    elev = m.acos(z/r)                          # theta
    az = m.atan2(y,x)                           # phi
    return r, elev, az

# Test mesh loading, saving & resizing

In [4]:
input_mesh = pymesh.load_mesh("meshes/sausage_shape.obj")

mesh = Mesh3D(input_mesh, CELL_SIZE)

pymesh.save_mesh("meshes/regularised_cell.ply", mesh.input_mesh)
mesh.write_input_mesh("meshes/regularised_cell.vtk")
mesh.write_mesh("meshes/regularised_cell_tetra.vtk")

# Test boundary node marking

In [5]:
tetmesh = mesh.tetmesh
boundary_mesh = pymesh.compute_outer_hull(mesh.tetmesh)

is_boundary = np.zeros(tetmesh.num_vertices)
source_faces = boundary_mesh.get_attribute("face_sources").astype(int)
for i_face in source_faces:
    face = tetmesh.faces[i_face]
    is_boundary[face] = True

mesh.set_attribute("is_boundary", is_boundary)

mesh.write_mesh("meshes/regularised_cell_tetra_with_boundary.vtk")

In [6]:
tetmesh = mesh.tetmesh
boundary_mesh = pymesh.compute_outer_hull(mesh.tetmesh)

is_boundary = np.zeros(tetmesh.num_vertices)
source_faces = boundary_mesh.get_attribute("face_sources").astype(int)
boundary_vertices = np.unique(np.array(tetmesh.faces[source_faces]).flatten())

is_boundary[boundary_vertices] = True

mesh.set_attribute("is_boundary", is_boundary)

mesh.write_mesh("meshes/regularised_cell_tetra_with_boundary.vtk")

# Test node data output

In [7]:
vertex_data = tetmesh.vertices[:, 2]
mesh.set_attribute("data", vertex_data)

mesh.write_mesh("meshes/regularised_cell_tetra_with_data.vtk")

# Test Laplace equation

In [8]:
def save_mesh(mesh, soln, analytical_fn, output_filename):
    analytical_soln = np.array([analytical_fn(v) for v in mesh.vertices()])

    # Calculate error
    error = np.abs(soln - analytical_soln)
    print("Error is", np.linalg.norm(error))

    # Save mesh + data
    mesh.set_attribute("soln", soln)
    mesh.set_attribute("error", error)
    mesh.set_attribute("analytical_soln", analytical_soln)

    mesh.write_mesh(output_filename)

In [9]:
def setup_boundaries(dirichlet_fn, von_neumann_fn,
                     dirichlet_boundary_vertices,
                     mesh, stiffness):
    # Setup von Neumann boundary conditions on RHS
    boundary_values = mesh.calculate_boundary_values(von_neumann_fn)
    von_neumann_rhs = mesh.assemble_von_neumann_boundary(boundary_values)
    
    # Setup dirichlet boundary conditions on RHS
    dirichlet_rhs = np.zeros(mesh.n_vertices)
    boundary_values = np.array(mesh.calculate_boundary_values(dirichlet_fn))
    dirichlet_rhs[dirichlet_boundary_vertices] = \
        boundary_values[dirichlet_boundary_vertices]

    # Remove dirichlet boundary points from stiffness mat
    stiffness[dirichlet_boundary_vertices,:] = 0.0
    # And ensure matrix isn't singular
    for vertex in dirichlet_boundary_vertices:
        stiffness[vertex, vertex] = 1.0
        
    return dirichlet_rhs, von_neumann_rhs

In [10]:
def run_steady_state(dirichlet_fn, von_neumann_fn, analytical_fn,
                     dirichlet_boundary_vertices,
                     output_filename, mesh):
    # Calculate stiffness mat for Laplacian operator
    stiffness = np.asarray(mesh.assembler.assemble("laplacian").todense())
    
    # Calulate boundary components of RHS
    dirichlet_rhs, von_neumann_rhs = setup_boundaries(dirichlet_fn, von_neumann_fn,
                     dirichlet_boundary_vertices,
                     mesh, stiffness)

    # Setup RHS
    rhs_vector = np.zeros(mesh.n_vertices)
    rhs_vector += dirichlet_rhs
    rhs_vector += von_neumann_rhs

    # Solve
    soln = np.linalg.solve(stiffness, rhs_vector)

    save_mesh(mesh, soln, analytical_fn, output_filename)

## Dirichlet 1

In [11]:
def calculate_dirichlet_values(vertex):
    # u = z around boundary
    return vertex[2]

def calc_analytical_solution(vertex):
    # Solution is globally u=z
    return vertex[2]

def calculate_von_neumann_values(vertex):
    return 0.0

output_filename = "meshes/laplace_equation_dirichlet_1_result.vtk"
                 
# Generate mesh from inbuilt sphere
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
mesh = Mesh3D(input_mesh, CELL_SIZE)

run_steady_state(calculate_dirichlet_values, calculate_von_neumann_values,
                calc_analytical_solution, mesh.boundary_vertices,
                output_filename, mesh)

Error is 1.1632857603153942e-17


## Dirichlet 2

In [12]:
def calculate_dirichlet_values(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    return m.cos(theta)**2

def calc_analytical_solution(vertex):
    r, theta, phi = cart2sph(vertex[0], vertex[1], vertex[2])
    c = [1, 0, 2*(r)**2]
    return 1/3*np.polynomial.legendre.legval(m.cos(theta), c)

def calculate_von_neumann_values(vertex):
    return 0.0

output_filename = "meshes/laplace_equation_dirichlet_2_result.vtk"
                 
# Generate mesh from inbuilt sphere
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
mesh = Mesh3D(input_mesh, CELL_SIZE)

run_steady_state(calculate_dirichlet_values, calculate_von_neumann_values,
                calc_analytical_solution, mesh.boundary_vertices,
                output_filename, mesh)

Error is 1.132892344728772e-15


## von Neumann

In [13]:
def calculate_dirichlet_values(vertex):
    return vertex[2]

def calc_analytical_solution(vertex):
    return vertex[2]

def calculate_von_neumann_values(vertex):
    return vertex[2]

output_filename = "meshes/laplace_equation_von_neumann_result.vtk"

# Generate mesh from inbuilt sphere
input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=4)
mesh = Mesh3D(input_mesh, CELL_SIZE)

dirichlet_boundary_vertices = np.array([mesh.boundary_vertices[0]])

run_steady_state(calculate_dirichlet_values, calculate_von_neumann_values,
                calc_analytical_solution, dirichlet_boundary_vertices,
                output_filename, mesh)

Error is 0.029249456401662367


# Test Heat Equation

In [14]:
def set_initial_conditions(mesh, calculate_initial_condition):
    return np.array(list(map(calculate_initial_condition, mesh.vertices())))

In [15]:
def run_heat_equation(dirichlet_fn, von_neumann_fn,
                      analytical_fn, ic_fn,
                     dirichlet_boundary_vertices,
                     output_filename, mesh, dt, T):
    # Calculate stiffness mat for Laplacian operator
    stiffness = np.asarray(mesh.assembler.assemble("laplacian").todense())
    mass = np.asarray(mesh.assembler.assemble("mass").todense())
    
    # Calulate boundary components of RHS
    dirichlet_rhs, von_neumann_rhs = setup_boundaries(dirichlet_fn, von_neumann_fn,
                     dirichlet_boundary_vertices,
                     mesh, stiffness)
    
    # Initial conditions
    u_prev = set_initial_conditions(mesh, ic_fn)

    # Solve
    lhs_matrix = dt*stiffness + mass

    t = 0
    while t<T:
        rhs_vector = mass @ u_prev + dt*(dirichlet_rhs + von_neumann_rhs)
        u_next = np.linalg.solve(lhs_matrix, rhs_vector)
        u_prev[:] = u_next[:]
        t+=dt

    soln = u_next

    save_mesh(mesh, soln, analytical_fn, output_filename)

## Dirichlet

In [16]:
def calculate_dirichlet_values(vertex):
    return 0.5

def calculate_initial_condition(vertex):
    return 0.0

def calc_analytical_solution(vertex):
    return 0.5

def calculate_von_neumann_values(vertex):
    return 0.0

output_filename = "meshes/heat_equation_dirichlet_result.vtk"

input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=2)
mesh = Mesh3D(input_mesh, CELL_SIZE)
    
dt = 0.1
T = 10.0

dirichlet_boundary_vertices = np.array(mesh.boundary_vertices)

run_heat_equation(calculate_dirichlet_values, calculate_von_neumann_values,
                      calc_analytical_solution, calculate_initial_condition,
                     dirichlet_boundary_vertices,
                     output_filename, mesh, dt, T)

Error is 1.4325749365178209e-15


## von Neumann

In [17]:
def calculate_dirichlet_values(vertex):
    return vertex[2]

def calculate_initial_condition(vertex):
    return 0.0

def calc_analytical_solution(vertex):
    return vertex[2]

def calculate_von_neumann_values(vertex):
    return vertex[2]

output_filename = "meshes/heat_equation_von_neumann_result.vtk"

input_mesh = pymesh.generate_icosphere(1, (0,0,0), refinement_order=3)
mesh = Mesh3D(input_mesh, CELL_SIZE)
    
dt = 0.1
T = 10.0

dirichlet_boundary_vertices = np.array([mesh.boundary_vertices[0]])

run_heat_equation(calculate_dirichlet_values, calculate_von_neumann_values,
                      calc_analytical_solution, calculate_initial_condition,
                     dirichlet_boundary_vertices,
                     output_filename, mesh, dt, T)

Error is 0.05694300953869292
